In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')  # Set whitegrid style from Seaborn
#import mplfinance as mpf
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import plotly.express as px
import matplotlib.ticker as tkr
import matplotlib.dates as mdates
from pathlib import Path

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

from collections import defaultdict
from scipy import stats

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)

#Keras
import tensorflow as tf

# keras keras

from tensorflow.python.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential, Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D, Embedding
from tensorflow.keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.layers import Reshape, Dropout, Dense,Multiply, Dot, Concatenate,Embedding
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences
from keras.models import Sequential
#from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN, Dropout, GRU, Bidirectional,Dense
from keras.layers import GlobalMaxPooling1D
from keras.models import Model, Sequential
from keras.layers import Embedding, LSTM,Dense
from keras.optimizers import SGD
from keras.layers import Input
from keras.layers import concatenate
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Time-series
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Modelling and Forecasting
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split

#KERAS GRID Search
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.activations import swish
#
import math
#

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
#
from datetime import datetime
from datetime import timedelta
#
# Load the model
from tensorflow.keras.models import load_model
#iimport pickle
import pickle

### FUNCTIONS

In [2]:
### FUNCTIONS
#Function to separate the DATA for LSTM
def createXY(dataset, n_lags):

    dataX, dataY = [], []

    for i in range(n_lags, len(dataset)):
            dataX.append(dataset[i - n_lags:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])

    return np.array(dataX),np.array(dataY)

In [3]:
#Defining Various Functions
#def mean_error(y_true, y_pred):
#   error = y_true - y_pred
#   return np.mean(error) 

def mean_error(y_true, y_pred):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    error_sum = 0
    for pred, true in zip(y_pred, y_true):
        error_sum += true + (-pred)
    return error_sum / len(y_true)


def mean_directional_accuracy(actual, predicted):
    """
    Calculates the Mean Directional Accuracy (MDA) for two time series.
    MDA measures the percentage of times the forecast and the actual values move in the same direction (i.e., both increase or both decrease).
    
    Parameters:
    actual (array-like): The actual values for the time series.
    predicted (array-like): The predicted values for the time series.
    
    Returns:
    float: The MDA value.
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    # Check if inputs are of the same length and non-empty
    if len(actual) != len(predicted):
        raise ValueError("The lengths of actual and predicted must be the same.")
    if len(actual) < 2:
        raise ValueError("The length of the time series must be at least 2 to compute MDA.")
    
    # Calculate the signs of the differences between consecutive values
    actual_diff = np.diff(actual)
    actual_signs = np.sign(actual_diff)
    predicted_diff = np.diff(predicted)
    predicted_signs = np.sign(predicted_diff)
    
    # Count the number of times the signs are the same
    num_correct = np.sum(actual_signs == predicted_signs)
    
    # Calculate the MDA value
    mda = num_correct / (len(actual) - 1)
    
    return mda
#
def directional_accuracy(actual, predicted):
    """
    Calculates the Directional Accuracy (DA) for two time series.
    
    Parameters:
    actual (array-like): The actual values for the time series.
    predicted (array-like): The predicted values for the time series.
    
    Returns:
    float: The DA value.
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    # Check if inputs are of the same length and non-empty
    if len(actual) != len(predicted):
        raise ValueError("The lengths of actual and predicted must be the same.")
    if len(actual) < 2:
        raise ValueError("The length of the time series must be at least 2 to compute DA.")
    
    # Calculate differences between consecutive values
    actual_diff = np.diff(actual)
    predicted_diff = np.diff(predicted)
    
    # Calculate the indicator function for directional accuracy
    indicator = (predicted_diff * actual_diff) > 0
    
    # Count the number of correct directions
    num_correct_directions = np.sum(indicator)
    
    # Calculate the DA value
    da = num_correct_directions / (len(actual) - 1)
    
    return da


In [4]:
# Define the evaluation function
def evaluation(y_test, y_pred, data_test,title):
    # Plot the actual vs predicted values
    fig = plt.figure(figsize=(8, 3))
    plt.plot(y_test, color='red', label='Actual')
    plt.plot(y_pred, color='blue', label='Predicted')
    #plt.xticks(np.arange(0, len(data_test), 100), data_test.index[np.arange(0, len(data_test), 100)], rotation=45)
    plt.title(f"Predictions vs Actual Data using {title}")
    #plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('EUR')
    plt.legend()
    plt.grid(False)

    # Assuming data_test.index is a datetime index
    plt.xticks(np.arange(0, len(data_test), 100), data_test.index[np.arange(0, len(data_test), 100)].strftime('%Y-%m-%d'), rotation=0)

    # Calculate metrics
    #mape = mean_absolute_percentage_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    me = mean_error(y_test, y_pred)
    mda = mean_directional_accuracy(y_test, y_pred)
    da  = directional_accuracy(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Round metrics for display
   # mape = np.round(mape, 3)
    mae = np.round(mae, 3)
    mse = np.round(mse, 3)
    rmse = np.round(rmse, 3)
    me = np.round(me, 3)
    mda = np.round(mda, 3)
    da= np.round(da, 3)
    r2 = np.round(r2, 3)

    # Print metrics
    print('MAE: %f  --  MSE: %f  --  RMSE: %f  --  ME: %f  --  MDA: %f  --  DA: %f  --  R2: %f' % 
          (mae, mse, rmse, me, mda, da, r2))

    return mae, mse, rmse, me, mda, da, r2

### LOAD TRAINED MODEL AND SCALER

In [5]:
#Load the model
model_gru = load_model('GRU_MODEL_KERAS.keras')
#Load the scaler
scaler = pickle.load(open('SCALERGRU.pickle', 'rb'))

### LOAD DATA WITH HOLIDAYS DAYS WEEKS MONTH IN SIN/COS. NEED TO USE NOTEBOOK CreateFeatureDataFrames

In [6]:
#Select the past 48 Hours_ These are the 48 LAGS to put into the model to find the next hour. LAST SEQUENCE of DATA
df_24 = pd.read_csv('PAST_24HOURS_SIN.csv')
df_24.drop(columns=['Unnamed: 0'],inplace=True, axis = 1)

In [7]:
df_24.head()

,datetime_from,price,price1,price2,price3,is_weekend,is_holiday,dayofweek_sin,dayofweek_cos,month_sin,month_cos,hour_sin,hour_cos
0,2024-07-01 00:00:00,86.07,92.98,102.76,84.455,0,0,0.0,1.0,1.224647e-16,-1.0,0.000000,1.000000
1,2024-07-01 01:00:00,90.00,90.37,101.96,84.156,0,0,0.0,1.0,1.224647e-16,-1.0,0.258819,0.965926
2,2024-07-01 02:00:00,92.53,93.08,101.86,84.687,0,0,0.0,1.0,1.224647e-16,-1.0,0.500000,0.866025
3,2024-07-01 03:00:00,104.00,96.77,106.27,91.973,0,0,0.0,1.0,1.224647e-16,-1.0,0.707107,0.707107
4,2024-07-01 04:00:00,140.25,110.12,118.50,109.203,0,0,0.0,1.0,1.224647e-16,-1.0,0.866025,0.500000


In [8]:
#LOAD the data for the next day with the Forecaster Predictions 
df_last_day = pd.read_csv('NEXTDAY24_FORECASTERS_SIN.csv')
df_last_day.drop(columns=['Unnamed: 0','price'],inplace=True, axis = 1)

In [9]:
df_24['datetime_from'] = pd.to_datetime(df_24['datetime_from'])
df_24 = df_24.set_index('datetime_from')
df_24 = df_24.asfreq('1H')
df_24 = df_24.sort_index()
(df_24.index == pd.date_range(start=df_24.index.min(),end=df_24.index.max(),freq=df_24.index.freq)).all()

True

In [10]:
df_last_day['datetime_from'] = pd.to_datetime(df_last_day['datetime_from'])
df_last_day = df_last_day.set_index('datetime_from')
df_last_day = df_last_day.asfreq('1H')
df_last_day = df_last_day.sort_index()
(df_last_day.index == pd.date_range(start=df_last_day.index.min(),end=df_last_day.index.max(),freq=df_last_day.index.freq)).all()

True

In [11]:
#Shape should be (24,12)
df_24.shape

(24, 12)

In [12]:
#Shape should be (24,11)
df_last_day.shape

(24, 11)

### DEFINE PARAMETERS

In [13]:
#Parameters
lag = 24
num_features = df_24.shape[1]
next_predictions = 24

### SCALE THE DATA of PAST OBSERVATIONS DF_24

In [14]:
#Scale the DATA with IMPORTED SCALER

data_scaled = scaler.transform(df_24.values)

#Need this to be (24,12)
data_scaled.shape

(24, 12)

### PREDICT NEXT 24 HOURS

In [15]:
#Model Expects an input of (1, 24, 12) to make a prediction of (1,)
#RESHAPE THE DATA (1, lag, num_features)
last_sequence = data_scaled.reshape((1, lag, num_features))  # (1, 24, 12)

In [17]:
# Initialize the predictions dictionary
predictions = {
    'pred': [],
    'price1': [],
    'price2': [],
    'price3': []
}

# Predict 24 hours ahead (next_predictions)
for i in range(next_predictions):
    # Predict the next hour
    next_prediction = model_gru.predict(last_sequence)
    
    # Inverse scale the prediction for the first feature
    prediction_reverselyscaled = scaler.inverse_transform(np.repeat(next_prediction, num_features, axis=-1))[:, 0]
    
    # Get the next timestamp
    next_timestamp = df_24.index[-1] + pd.Timedelta(hours=i + 1)
    #print('next_timestamp: ',next_timestamp)
    
    # Extract the next feature values from df_last_day for the next timestamp
    if next_timestamp in df_last_day.index:
        next_features = df_last_day.loc[next_timestamp, ['price1', 'price2', 'price3', 'is_weekend', 'is_holiday', 'dayofweek_sin', 'dayofweek_cos', 'month_sin', 'month_cos', 'hour_sin', 'hour_cos']].values
    else:
        next_features = np.zeros(num_features - 1)  # Default to zeros if timestamp is out of range
        print('Timestamp not in Next Day !!!')
    
    # Combine the predicted value with the next feature values
    new_row = np.concatenate(([prediction_reverselyscaled[0]], next_features))
    # Scale the new row
    new_row_scaled = scaler.transform(new_row.reshape(1, -1)).flatten()

    # Update the last_sequence with the new prediction and feature values
    next_input = np.append(last_sequence[0, 1:, :], [new_row_scaled], axis=0)  # Shift and append the prediction
    last_sequence = next_input.reshape((1, lag, num_features))
    
    # Update the predictions dictionary
    predictions['pred'].append(prediction_reverselyscaled[0])
    if next_timestamp in df_last_day.index:
        predictions['price1'].append(df_last_day.loc[next_timestamp, 'price1'])
        predictions['price2'].append(df_last_day.loc[next_timestamp, 'price2'])
        predictions['price3'].append(df_last_day.loc[next_timestamp, 'price3'])
    else:
        predictions['price1'].append(None) # Append None if the timestamp is out of range
        predictions['price2'].append(None)
        predictions['price3'].append(None)
    
    # Debugging: Print the current prediction and last sequence
    #print(f"Prediction {i+1}: {predictions['pred'][-1]}")
    #print(f"Actual {i+1}: {predictions['actual'][-1]}")
    #print(f"last_sequence before update: {last_sequence[0, -1, :]}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


In [18]:
# Print the 24-hour ahead predictions
print("24-hour ahead predictions:")
print(print('Predicted Prices: ',predictions['pred']),print('Price1: ',predictions['price1']),print('Price2: ',predictions['price2']), print('Price3: ',predictions['price3']))

24-hour ahead predictions:
Predicted Prices:  [104.92036, 102.64738, 101.78009, 106.29398, 117.32009, 134.73607, 143.774, 128.66585, 114.02026, 102.88171, 95.32001, 92.38086, 93.3325, 95.7201, 109.326866, 123.21036, 149.39859, 206.14989, 265.15945, 219.73792, 157.10446, 129.66829, 118.57099, 107.275665]
Price1:  [121.26, 114.02, 115.74, 120.53, 139.18, 141.21, 132.88, 121.5, 110.53, 105.25, 106.19, 109.59, 109.01, 123.32, 138.23, 185.26, 251.67, 290.56, 287.71, 226.52, 163.51, 134.43, 96.33, 93.89]
Price2:  [104.18, 103.18, 102.94, 106.32, 119.03, 125.14, 113.0, 95.66, 86.53, 81.18, 81.58, 87.06, 92.54, 100.39, 116.62, 114.67, 155.64, 239.54, 278.15, 209.92, 137.36, 115.99, 110.49, 101.37]
Price3:  [95.612, 93.174, 93.678, 104.669, 122.26, 149.214, 128.035, 105.012, 87.162, 78.436, 73.386, 73.735, 75.004, 92.217, 100.722, 110.045, 138.43, 219.046, 278.24, 209.134, 137.654, 116.154, 97.723, 97.265]
None None None None
